In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:

! wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
! chmod +x Anaconda3-2019.10-Linux-x86_64.sh
! bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local

# ! wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh 
# ! chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh
# ! time bash Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /usr/local

! time conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

!pip install mordred
!pip install --upgrade bravado
!pip install molvs

!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

--2021-04-29 02:58:44--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M   106MB/s    in 4.8s    

2021-04-29 02:58:49 (106 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.

In [3]:
!wget https://raw.githubusercontent.com/jm5159/E4511-2021-Mao/master/Dataset/data_for_model_MAOA.csv

--2021-04-29 03:06:14--  https://raw.githubusercontent.com/jm5159/E4511-2021-Mao/master/Dataset/data_for_model_MAOA.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 355064 (347K) [text/plain]
Saving to: ‘data_for_model_MAOA.csv’

data_for_model_MAOA 100%[===================>] 346.74K  --.-KB/s    in 0.02s   

2021-04-29 03:06:15 (15.5 MB/s) - ‘data_for_model_MAOA.csv’ saved [355064/355064]



In [4]:
!pip install hyperopt

     |████████████████████████████████| 972kB 9.4MB/s 


In [5]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_context("poster")
sns.set_style("whitegrid")
sns.set_palette("Set2")

import pandas as pd
import os, json, sys, glob, pickle

from atomsci.ddm.pipeline import parameter_parser as parse
import atomsci.ddm.pipeline.model_pipeline as mp
from atomsci.ddm.pipeline import perf_data

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
h1 = pd.read_csv("data_for_model_MAOA.csv")

In [8]:
h1=h1.iloc[:,2:]

In [9]:
h1

,compound_id,base_rdkit_smiles,relation,pXC50,active,VALUE_NUM_std,Perc_Var,Remove_BadDuplicate,VALUE_NUM_mean
0,CHEMBL248049,CC1CCC(=NNc2nc(-c3ccc(Cl)cc3)cs2)CC1,NaN,4.818836,0,NaN,0.0,0,4.818836
1,CHEMBL3942207,N#Cc1ccccc1S(=O)(=O)Nc1cccc(-c2ncc([C@H]3C[C@@...,NaN,4.000000,0,NaN,0.0,0,4.000000
2,CHEMBL3651121,Clc1ncccc1CN[C@H]1CC1c1ccccc1,NaN,3.472120,0,NaN,0.0,0,3.472120
3,CHEMBL4206777,NC(=O)c1ccc(OCc2cccc(C(F)(F)F)c2)cc1,<,4.000000,0,NaN,0.0,0,4.000000
4,CHEMBL3771110,CC(=O)c1cccc(OCc2ccc(Cl)cc2)c1,NaN,4.401895,0,NaN,0.0,0,4.401895
...,...,...,...,...,...,...,...,...,...
3539,CHEMBL1830793,CC(=O)c1cc2cc(O)ccc2oc1=O,NaN,3.820000,0,NaN,0.0,0,3.820000
3540,CHEMBL3218145,c1ccc2cc(-c3csc(NN=C4CCCCC4)n3)ccc2c1,NaN,5.703335,0,NaN,0.0,0,5.703335
3541,CHEMBL3330342,N[C@@]1(Cc2ccccc2)C[C@@H]1c1ccccc1,NaN,5.080399,0,NaN,0.0,0,5.080399
3542,CHEMBL3667543,Cn1ccc(-c2ccc(CNC(=O)c3c(Cl)cccc3Cl)cc2)cc1=O,<,3.886057,0,NaN,0.0,0,3.886057


In [10]:
train_file = "data_for_model_MAOA.csv"
response_col = "active"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"

params = {
        "system": "LC",
        "lc_account": 'None',
        "datastore": "False",
        "save_results": "False",
        "data_owner": "username",
        "prediction_type": "classification",
        "dataset_key": train_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_only": "True",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": "split"
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2021-04-29 03:08:53,833 Splitting data by scaffold


number of features: 1024


2021-04-29 03:08:56,725 Dataset split table saved to /data_for_model_MAOA_train_valid_test_scaffold_f22adf81-f5b6-4b42-9929-59c8d6c47547.csv


In [11]:
listing = glob.glob(os.path.join("/", "data_for_model_MAOA_train_valid_test_scaffold_*"))
name = listing[0]
split_uuid = name.split("_")[-1].split(".")[0]
response_col = "active"

params = {
        "system": "LC",
        "lc_account": 'None',
        "datastore": "False",
        "save_results": "False",
        "data_owner": "username",
        "prediction_type": "classification",
        "dataset_key": train_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "split_uuid": split_uuid,
        "previously_split": "True",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": "ampl_output"
    }

In [12]:
ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

number of features: 1024


2021-04-29 03:10:11,160 Previous dataset split restored
2021-04-29 03:10:19,951 Wrote model metadata to file ampl_output/data_for_model_MAOA/RF_ecfp_scaffold_classification/04abea6b-ab28-4a57-a11a-60f1a44e634c/model_metadata.json
2021-04-29 03:10:20,094 Wrote model metrics to file ampl_output/data_for_model_MAOA/RF_ecfp_scaffold_classification/04abea6b-ab28-4a57-a11a-60f1a44e634c/model_metrics.json


Wrote model tarball to ampl_output/data_for_model_MAOA_model_04abea6b-ab28-4a57-a11a-60f1a44e634c.tar.gz


In [13]:
pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results = pred_data.get_prediction_results()
print(f"AUC score of validation set: {pred_results['roc_auc_score']:.3f}")

AUC score of validation set: 0.829


In [14]:
estimators = [200]
depth=[70]
features=[10]

rfe_list = []
rfd_list = []
rff_list = []
valid_auc_list = []
for rf_estimator in estimators:
    for rf_depth in depth:
        for rf_feature in features:
            params["rf_estimators"] = rf_estimator
            params["rf_max_depth"] = rf_depth
            params["rf_max_features"] = rf_feature
            rfe_list.append(rf_estimator)
            rfd_list.append(rf_depth)
            rff_list.append(rf_feature)
            tp = parse.wrapper(params)
            pl = mp.ModelPipeline(tp)
            pl.train_model()
            pred_data = pl.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
            pred_results = pred_data.get_prediction_results()
            valid_auc = pred_results['roc_auc_score']
            valid_auc_list.append(valid_auc)
            print(f"rf_estimators: {rf_estimator}, rf_max_depth: {rf_depth}, rf_max_features: {rf_feature}, valid_auc: {valid_auc}")

number of features: 1024


2021-04-29 03:11:24,248 Previous dataset split restored
2021-04-29 03:11:27,549 Wrote model metadata to file ampl_output/data_for_model_MAOA/RF_ecfp_scaffold_classification/f70f2664-4b78-42c9-a4c5-68432ada213a/model_metadata.json
2021-04-29 03:11:27,648 Wrote model metrics to file ampl_output/data_for_model_MAOA/RF_ecfp_scaffold_classification/f70f2664-4b78-42c9-a4c5-68432ada213a/model_metrics.json


Wrote model tarball to ampl_output/data_for_model_MAOA_model_f70f2664-4b78-42c9-a4c5-68432ada213a.tar.gz
rf_estimators: 200, rf_max_depth: 70, rf_max_features: 10, valid_auc: 0.8400252525252525


In [15]:
!wget https://raw.githubusercontent.com/fanroyi/E4511-2021-Fan/main/Dataset/curated_combined_df.csv

--2021-04-29 03:12:16--  https://raw.githubusercontent.com/fanroyi/E4511-2021-Fan/main/Dataset/curated_combined_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 633765 (619K) [text/plain]
Saving to: ‘curated_combined_df.csv’

curated_combined_df 100%[===================>] 618.91K  --.-KB/s    in 0.03s   

2021-04-29 03:12:16 (17.4 MB/s) - ‘curated_combined_df.csv’ saved [633765/633765]



In [17]:
fan=pd.read_csv('curated_combined_df.csv',index_col=0)
fan_no_overlap=fan[~fan.base_rdkit_smiles.isin(h1.base_rdkit_smiles)]

In [18]:
results_df = pl.predict_on_dataframe(fan_no_overlap, contains_responses=True)
results_df

2021-04-29 03:13:40,198 Featurizing data...
2021-04-29 03:14:23,847 Done


number of features: 1024


,base_rdkit_smiles,actual,pred
compound_id,,,
JOJYHYRCIYAVHN-UHFFFAOYSA-N,Cc1ccc(Sc2ccccc2N2CCNCC2)c(C)c1,0.0,0
HHVBUSGGPUQAOP-UHFFFAOYSA-N,Cc1cccc(Sc2ccccc2N2CCNCC2)c1C,0.0,0
RJOWFEDJGNNCDF-UHFFFAOYSA-N,Clc1ccc(Sc2ccccc2N2CCNCC2)c(Cl)c1,0.0,0
XUMDENDMRHWGGR-UHFFFAOYSA-N,Clc1ccc(Sc2ccccc2N2CCNCC2)cc1,0.0,0
VHKBDODTXKHVJR-UHFFFAOYSA-N,Cc1ccc(Sc2ccccc2N2CCNCC2)cc1C,0.0,0
...,...,...,...
CHEMBL1372952,COc1ccc2[nH]c(Sc3c([N+](=O)[O-])nc(C)n3Cc3cccc...,0.0,0
CHEMBL1335370,O=C(CSc1ccccc1)Nc1ccc(N2CCN(c3ccccc3)CC2)c(F)c1,1.0,0
CHEMBL103,CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4CC(=O)CC[C@]4(...,1.0,0


In [19]:
from sklearn.metrics import roc_auc_score
y_true = results_df['actual'].values
y_pred = results_df['pred'].values
roc_auc_score(y_true, y_pred)

0.5000091461683288

In [20]:
!wget https://raw.githubusercontent.com/js5783/E4511-2021-Shen/main/Dataset/delaney_curated_final.csv

--2021-04-29 03:15:21--  https://raw.githubusercontent.com/js5783/E4511-2021-Shen/main/Dataset/delaney_curated_final.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2231626 (2.1M) [text/plain]
Saving to: ‘delaney_curated_final.csv’

delaney_curated_fin 100%[===================>]   2.13M  --.-KB/s    in 0.05s   

2021-04-29 03:15:22 (45.0 MB/s) - ‘delaney_curated_final.csv’ saved [2231626/2231626]



In [21]:
shen=pd.read_csv('delaney_curated_final.csv')
shen_no_overlap=shen[~shen.base_rdkit_smiles.isin(h1.base_rdkit_smiles)]
results_df = pl.predict_on_dataframe(shen_no_overlap, contains_responses=True)
results_df

2021-04-29 03:16:07,813 Featurizing data...
2021-04-29 03:17:12,257 Done


number of features: 1024


,base_rdkit_smiles,actual,pred
compound_id,,,
CHEMBL245412,C/C1=C\CC(C)(C)/C=C/C(=O)/C(C)=C/CC1,0.0,0
CHEMBL1419469,Cc1ccccc1NC(=O)Cc1c(O)nc2ccccc2c1O,0.0,0
CHEMBL1093607,CC(c1c(CCN(C)C)sc2ccccc12)n1ccnc1,0.0,0
CHEMBL66654,Cc1cccc(C#Cc2ccccc2)n1,0.0,0
CHEMBL1091782,C[C@@H](c1cscn1)c1c(CCN(C)C)sc2ccccc12,0.0,0
...,...,...,...
CHEMBL1089116,CCn1cc(-c2ccnc3[nH]ccc23)c(-c2ccc(NC(=O)Nc3ccc...,1.0,0
CHEMBL1081870,Cc1cccc(C)c1CNc1nccc2c1nc(C)n2C,1.0,0
53392850,CN1C[C@H]2CCCC[C@@]2(Cc2ccc(Cl)c(Cl)c2)C1,1.0,0


In [22]:
from sklearn.metrics import roc_auc_score
y_true = results_df['actual'].values
y_pred = results_df['pred'].values
roc_auc_score(y_true, y_pred)

0.499953934033536

In [23]:
!wget https://raw.githubusercontent.com/jm5159/E4511-2021-Mao/master/Dataset/CYP3A4_cur_final.csv

--2021-04-29 03:17:39--  https://raw.githubusercontent.com/jm5159/E4511-2021-Mao/master/Dataset/CYP3A4_cur_final.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3199248 (3.1M) [text/plain]
Saving to: ‘CYP3A4_cur_final.csv’

CYP3A4_cur_final.cs 100%[===================>]   3.05M  --.-KB/s    in 0.1s    

2021-04-29 03:17:40 (22.8 MB/s) - ‘CYP3A4_cur_final.csv’ saved [3199248/3199248]



In [24]:
mao=pd.read_csv('CYP3A4_cur_final.csv')
mao_no_overlap=mao[~mao.base_rdkit_smiles.isin(h1.base_rdkit_smiles)]
results_df = pl.predict_on_dataframe(mao_no_overlap, contains_responses=True)
results_df

2021-04-29 03:18:29,567 Featurizing data...
2021-04-29 03:20:16,777 Done


number of features: 1024


,base_rdkit_smiles,actual,pred
compound_id,,,
CHEMBL1537880,Cc1ccc(-c2nnc(C)c(N3CC(C)OC(C)C3)n2)cc1,0.0,0
CHEMBL1515033,CS(=O)(=O)N1CCC[C@@]2(CCN(c3ccncc3)C2)C1,0.0,0
CHEMBL3422242,Cc1cc([C@H]2C[C@H]3[C@@H](C)SC(N)=N[C@@]3(c3cc...,0.0,0
CHEMBL1526043,N#Cc1cccc(-c2nc3cnc(N4CCOCC4)nc3n(C3CC3)c2=O)c1,0.0,0
CHEMBL3145019,Cc1nn(C(C)(C)C)c(O)c1Sc1ccc(Cl)cc1,0.0,0
...,...,...,...
CHEMBL3932169,CC(C)[C@@H]1c2nn(-c3cccc(S(C)(=O)=O)c3)cc2CN1c...,0.0,0
CHEMBL4210866,N[C@@H]1CCC=C(c2ccc(F)cc2)[C@H]1CNc1cc(F)c(S(=...,0.0,0
CHEMBL1465534,O=[N+]([O-])c1cccc(-c2nnn(C34CC5CC(CC(C5)C3)C4...,0.0,0


In [25]:
from sklearn.metrics import roc_auc_score
y_true = results_df['actual'].values
y_pred = results_df['pred'].values
roc_auc_score(y_true, y_pred)

0.49991723224631684